##### <p> Samuel Wolfe <br> October 7, 2023 <br> MSBA 207 <br> Chapter 10b </p>

In [1]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mord import LogisticIT
import matplotlib.pylab as plt
import seaborn as sns
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
import math
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.width', 250)

In [2]:
DATA = Path('E:/Aliit/School/MSBA/206/MSBA-206/dmba')
Homework = Path('E:/Aliit/School/MSBA/207/MSBA-207/Homework')

ebay_df = pd.read_csv(DATA / 'eBayAuctions.csv')
ebay_df.columns = [c.replace('?','') for c in ebay_df.columns]
print(ebay_df)
print(ebay_df.dtypes)

              Category currency  sellerRating  Duration endDay  ClosePrice  OpenPrice  Competitive
0     Music/Movie/Game       US          3249         5    Mon        0.01       0.01            0
1     Music/Movie/Game       US          3249         5    Mon        0.01       0.01            0
2     Music/Movie/Game       US          3249         5    Mon        0.01       0.01            0
3     Music/Movie/Game       US          3249         5    Mon        0.01       0.01            0
4     Music/Movie/Game       US          3249         5    Mon        0.01       0.01            0
...                ...      ...           ...       ...    ...         ...        ...          ...
1967        Automotive       US          2992         5    Sun      359.95     359.95            0
1968        Automotive       US            21         5    Sat      610.00     300.00            1
1969        Automotive       US          1400         5    Mon      549.00     549.00            0
1970      

In [3]:
table1 = pd.pivot_table(ebay_df, index='Category', values='Competitive', aggfunc=[np.mean]).sort_values(by=[('mean','Competitive')])
table1

,mean
,Competitive
Category,
Health/Beauty,0.171875
EverythingElse,0.235294
Coins/Stamps,0.297297
Pottery/Glass,0.350000
Automotive,0.353933
Jewelry,0.365854
Books,0.500000
Clothing/Accessories,0.504202


In [4]:
table2 = pd.pivot_table(ebay_df, index='currency', values='Competitive', aggfunc=[np.mean]).sort_values(by=[('mean','Competitive')])
table2

,mean
,Competitive
currency,
US,0.519350
EUR,0.551595
GBP,0.687075


In [5]:
table3 = pd.pivot_table(ebay_df, index='Duration', values='Competitive', aggfunc=[np.mean]).sort_values(by=[('mean','Competitive')])
table3

,mean
,Competitive
Duration,
3,0.450704
7,0.489142
1,0.521739
10,0.544554
5,0.686695


In [6]:
table4 = pd.pivot_table(ebay_df, index='endDay', values='Competitive', aggfunc=[np.mean]).sort_values(by=[('mean','Competitive')])
table4

,mean
,Competitive
endDay,
Sat,0.427350
Fri,0.466899
Wed,0.480000
Sun,0.485207
Tue,0.532164
Thu,0.603960
Mon,0.673358


In [7]:
ebay_df.replace(['Sun','Wed','Fri'],'SWF', inplace=True)
ebay_df.replace(['Business/Industrial','Computer','Home/Garden'],'BICH', inplace=True)
ebay_df.replace(['Antique/Art/Craft', 'Collectibles'],'Collectibles', inplace=True)
ebay_df.replace(['Automotive','Pottery/Glass'],'AutoPotteryGlass', inplace=True)
ebay_df.replace(['Books','Clothing/Accessories'],'BooksClothing', inplace=True)
ebay_df

,Category,currency,sellerRating,Duration,endDay,ClosePrice,OpenPrice,Competitive
0,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
1,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
2,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
3,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
4,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
...,...,...,...,...,...,...,...,...
1967,AutoPotteryGlass,US,2992,5,SWF,359.95,359.95,0
1968,AutoPotteryGlass,US,21,5,Sat,610.00,300.00,1
1969,AutoPotteryGlass,US,1400,5,Mon,549.00,549.00,0
1970,AutoPotteryGlass,US,57,7,SWF,820.00,650.00,1


In [8]:
table5 = pd.pivot_table(ebay_df, index='Category', values='Competitive', aggfunc=[np.mean]).sort_values(by=[('mean','Competitive')])
table5

,mean
,Competitive
Category,
Health/Beauty,0.171875
EverythingElse,0.235294
Coins/Stamps,0.297297
AutoPotteryGlass,0.353535
Jewelry,0.365854
BooksClothing,0.502890
Toys/Hobbies,0.529915
Collectibles,0.572115


In [9]:
table6 = pd.pivot_table(ebay_df, index='endDay', values='Competitive', aggfunc=[np.mean]).sort_values(by=[('mean','Competitive')])
table6

,mean
,Competitive
endDay,
Sat,0.427350
SWF,0.477143
Tue,0.532164
Thu,0.603960
Mon,0.673358


In [10]:
ebay_df['Category'] = ebay_df['Category'].astype('category')
ebay_df['currency'] = ebay_df['currency'].astype('category')
ebay_df['Duration'] = ebay_df['Duration'].astype('category')
ebay_df['endDay'] = ebay_df['endDay'].astype('category')
print(ebay_df.dtypes)

Category        category
currency        category
sellerRating       int64
Duration        category
endDay          category
ClosePrice       float64
OpenPrice        float64
Competitive        int64
dtype: object


In [11]:
ebay_df_dum = pd.get_dummies(ebay_df, prefix_sep='_', drop_first=True, dtype='uint8')
print(ebay_df_dum.dtypes)

sellerRating                   int64
ClosePrice                   float64
OpenPrice                    float64
Competitive                    int64
Category_BICH                  uint8
Category_BooksClothing         uint8
Category_Coins/Stamps          uint8
Category_Collectibles          uint8
Category_Electronics           uint8
Category_EverythingElse        uint8
Category_Health/Beauty         uint8
Category_Jewelry               uint8
Category_Music/Movie/Game      uint8
Category_Photography           uint8
Category_SportingGoods         uint8
Category_Toys/Hobbies          uint8
currency_GBP                   uint8
currency_US                    uint8
Duration_3                     uint8
Duration_5                     uint8
Duration_7                     uint8
Duration_10                    uint8
endDay_SWF                     uint8
endDay_Sat                     uint8
endDay_Thu                     uint8
endDay_Tue                     uint8
dtype: object


In [12]:
y = ebay_df_dum['Competitive']
X = ebay_df_dum.drop(columns=['Competitive'])


In [13]:
# partition data
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)
train_X_all = train_X
train_X_less = train_X.drop(columns='ClosePrice')
valid_X_all = valid_X
valid_X_less = valid_X.drop(columns='ClosePrice')

In [14]:
train_X.to_csv(Homework / 'ebay_train_X.csv', index=False)
valid_X.to_csv(Homework / 'ebay_valid_X.csv', index=False)
train_y.to_csv(Homework / 'ebay_train_y.csv', index=False)
valid_y.to_csv(Homework / 'ebay_valid_y.csv', index=False)

In [15]:
train_X_all1 = sm.add_constant(train_X_all, prepend=True)
train_X_less1 = sm.add_constant(train_X_less, prepend=True)
valid_X_all1 = sm.add_constant(valid_X_all, prepend=True)
valid_X_less1 = sm.add_constant(valid_X_less, prepend=True)
#valid_X1 = sm.add_constant(valid_X, prepend=True)

In [16]:
logit_full1 = sm.GLM(train_y, train_X_all1, family=sm.families.Binomial())
logit_result1 = logit_full1.fit()
logit_result1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            Competitive   No. Observations:                 1183
Model:                            GLM   Df Residuals:                     1157
Model Family:                Binomial   Df Model:                           25
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -598.21
Date:                Sat, 07 Oct 2023   Deviance:                       1196.4
Time:                        12:37:54   Pearson chi2:                 4.48e+08
No. Iterations:                    22   Pseudo R-squ. (CS):             0.3098
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.8704      0.960     -0.906      0.365      -2.752       1.012
sellerRating              -4.426e-05   1.63e-05     -2.717      0.007   -7.62e-05   -1.23e-05
ClosePrice                    0.0828      0.009      9.507      0.000       0.066       0.100
OpenPrice                    -0.0996      0.011     -9.322      0.000      -0.120      -0.079
Category_BICH                 0.6586      0.383      1.718      0.086      -0.093       1.410
Category_BooksClothing       -0.1224      0.388     -0.315      0.753      -0.883       0.639
Category_Coins/Stamps        -1.4822      0.714     -2.075      0.038      -2.882      -0.082
Category_Collectibles         0.5921      0.322      1.838      0.066      -0.039       1.224
Category_Electronics          1.0679      0.594      1.799      0.072      -0.095       2.231
Category_EverythingElse      -1.0711      0.876     -1.222      0.222      -2.789       0.647
Category_Health/Beauty       -1.4084      0.556     -2.532      0.011      -2.498      -0.318
Category_Jewelry              0.1599      0.467      0.342      0.732      -0.756       1.075
Category_Music/Movie/Game     0.6313      0.324      1.946      0.052      -0.005       1.267
Category_Photography         22.2051   1.74e+04      0.001      0.999   -3.41e+04    3.41e+04
Category_SportingGoods        0.4739      0.459      1.032      0.302      -0.426       1.374
Category_Toys/Hobbies         0.7800      0.343      2.277      0.023       0.109       1.451
currency_GBP                  2.0350      0.575      3.537      0.000       0.907       3.162
currency_US                   0.6572      0.249      2.641      0.008       0.169       1.145
Duration_3                    0.0397      0.897      0.044      0.965      -1.719       1.799
Duration_5                    0.4650      0.889      0.523      0.601      -1.278       2.208
Duration_7                    0.1464      0.874      0.168      0.867      -1.567       1.860
Duration_10                   0.3555      0.902      0.394      0.693      -1.411       2.122
endDay_SWF                   -0.5807      0.209     -2.775      0.006      -0.991      -0.171
endDay_Sat                   -0.8850      0.260     -3.398      0.001      -1.396      -0.375
endDay_Thu                   -1.0467      0.532     -1.968      0.049      -2.089      -0.004
endDay_Tue                   -0.5035      0.287     -1.756      0.079      -1.066       0.059
=============================================================================================
"""

In [17]:
pred_all = logit_result1.predict(valid_X_all1)
predictions_all = [0 if x < 0.5 else 1 for x in pred_all]
classificationSummary(valid_y, predictions_all)

Confusion Matrix (Accuracy 0.7554)

       Prediction
Actual   0   1
     0 283  70
     1 123 313


#### Closing price has a coef of 0.0828 which because that is positive means higher closing prices increase the probability of an auction being competitive.

In [18]:
#odds
math.exp(0.0828)

1.0863245219159159

#### With every 1 dollar increase of closing price the chance of competitive goes up by a factor of 1. We cant use ClosePrice for live data so we cannot use it. Removing for next question.

In [19]:
logit_full2 = sm.GLM(train_y, train_X_less1, family=sm.families.Binomial())
logit_result2 = logit_full2.fit()
logit_result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            Competitive   No. Observations:                 1183
Model:                            GLM   Df Residuals:                     1158
Model Family:                Binomial   Df Model:                           24
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -718.92
Date:                Sat, 07 Oct 2023   Deviance:                       1437.8
Time:                        12:37:54   Pearson chi2:                 1.19e+03
No. Iterations:                    21   Pseudo R-squ. (CS):             0.1535
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         0.6057      0.729      0.831      0.406      -0.823       2.034
sellerRating              -4.508e-05   1.36e-05     -3.310      0.001   -7.18e-05   -1.84e-05
OpenPrice                    -0.0043      0.003     -1.577      0.115      -0.010       0.001
Category_BICH                 1.1373      0.321      3.543      0.000       0.508       1.766
Category_BooksClothing        0.4218      0.319      1.324      0.185      -0.203       1.046
Category_Coins/Stamps        -1.3911      0.617     -2.256      0.024      -2.600      -0.183
Category_Collectibles         0.8707      0.266      3.275      0.001       0.350       1.392
Category_Electronics          1.5637      0.484      3.229      0.001       0.615       2.513
Category_EverythingElse      -0.7589      0.737     -1.030      0.303      -2.203       0.685
Category_Health/Beauty       -1.4503      0.511     -2.838      0.005      -2.452      -0.449
Category_Jewelry              0.1775      0.418      0.425      0.671      -0.642       0.997
Category_Music/Movie/Game     0.5533      0.275      2.010      0.044       0.014       1.093
Category_Photography         23.0803   1.65e+04      0.001      0.999   -3.23e+04    3.23e+04
Category_SportingGoods        1.8243      0.390      4.682      0.000       1.061       2.588
Category_Toys/Hobbies         0.8401      0.293      2.863      0.004       0.265       1.415
currency_GBP                  1.4959      0.480      3.119      0.002       0.556       2.436
currency_US                   0.0553      0.211      0.262      0.793      -0.359       0.470
Duration_3                   -0.4202      0.675     -0.623      0.534      -1.743       0.902
Duration_5                    0.0077      0.669      0.012      0.991      -1.303       1.318
Duration_7                   -0.4966      0.654     -0.759      0.448      -1.779       0.786
Duration_10                  -0.4914      0.682     -0.720      0.471      -1.828       0.846
endDay_SWF                   -0.8433      0.192     -4.395      0.000      -1.219      -0.467
endDay_Sat                   -1.0160      0.235     -4.326      0.000      -1.476      -0.556
endDay_Thu                   -1.1759      0.432     -2.720      0.007      -2.023      -0.329
endDay_Tue                   -0.7178      0.265     -2.714      0.007      -1.236      -0.199
=============================================================================================
"""

In [20]:
pred_all = logit_result2.predict(valid_X_less1)
predictions_all = [0 if x < 0.5 else 1 for x in pred_all]
classificationSummary(valid_y, predictions_all)

Confusion Matrix (Accuracy 0.6502)

       Prediction
Actual   0   1
     0 206 147
     1 129 307


#### Without using ClosePrice the accuracy of our model goes down significantly. By 10.52%.

In [21]:
# Initiate scaler
sc = StandardScaler()
# Standardize the training dataset
train_X_transformed = pd.DataFrame(sc.fit_transform(train_X.drop(columns='ClosePrice')),index=train_X.drop(columns='ClosePrice').index, columns=train_X.drop(columns='ClosePrice').columns)
# Standardized the testing dataset
valid_X_transformed = pd.DataFrame(sc.transform(valid_X.drop(columns='ClosePrice')),index=valid_X.drop(columns='ClosePrice').index, columns=valid_X.drop(columns='ClosePrice').columns)

In [22]:
logit_reg_CV_L1 = LogisticRegressionCV(Cs=20, max_iter=10000, penalty='l1', solver='liblinear', cv=5, random_state=1, tol=1e-08)
logit_reg_CV_L1.fit(train_X_transformed, train_y)

LogisticRegressionCV(Cs=20, cv=5, max_iter=10000, penalty='l1', random_state=1,
                     solver='liblinear', tol=1e-08)

In [26]:
pd.set_option('display.width',20)
pd.set_option('display.precision',4)
pd.set_option('display.max_columns',40)
print('regularization', logit_reg_CV_L1.C_)
print('intercept', logit_reg_CV_L1.intercept_[0])
print(pd.DataFrame({'coeff': logit_reg_CV_L1.coef_[0]}, index=train_X.drop(columns='ClosePrice').columns).transpose())
pd.reset_option('display.width')
print('AIC', AIC_score(valid_y, logit_reg_CV_L1.predict(valid_X.drop(columns='ClosePrice')), df = len(train_X.drop(columns='ClosePrice').columns) +1))
pd.reset_option('display.width')
pd.reset_option('display.precision')
pd.reset_option('display.max_columns')

regularization [11.28837892]
intercept 0.16187419181310084
       sellerRating  \
coeff       -0.2765   

       OpenPrice  \
coeff     -0.189   

       Category_BICH  \
coeff         0.3079   

       Category_BooksClothing  \
coeff                  0.1168   

       Category_Coins/Stamps  \
coeff                -0.1925   

       Category_Collectibles  \
coeff                 0.3539   

       Category_Electronics  \
coeff                 0.244   

       Category_EverythingElse  \
coeff                   -0.073   

       Category_Health/Beauty  \
coeff                 -0.2752   

       Category_Jewelry  \
coeff            0.0309   

       Category_Music/Movie/Game  \
coeff                     0.2191   

       Category_Photography  \
coeff                0.6144   

       Category_SportingGoods  \
coeff                  0.4552   

       Category_Toys/Hobbies  \
coeff                 0.2581   

       currency_GBP  \
coeff        0.3992   

       currency_US  \
coeff       0.02